In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# List of url's archived in the wayback machine.

In [ ]:
import requests
items = []
initial = "http://archive.org/wayback/available"
# iterate through list of flagged twitter screen names
with open('/content/drive/MyDrive/Present work/2/twitter_handle_urls.csv') as f:
    for line in f:
        params = {'url': line}
        r = requests.get(initial, params=params)
        d = r.json()
        print(d)
        items.append(d)
# write URL of any available archives to file
with open('/content/drive/MyDrive/Present work/2/avail_urls.txt', 'w') as f:
    for item in items:
        if 'archived_snapshots' in item:
            if 'closest' in item['archived_snapshots']:
                f.write(item['archived_snapshots']['closest']['url'] + '\n')

{'url': 'http://twitter.com/10_gop\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/1488reasons\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/1D_Nicole_\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/1DRussianFNDM\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/1Erik_Lee\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/1lorenafava1\n', 'archived_snapshots': {}}
{'url': 'http://twitter.com/2lauragibson\n', 'archived_snapshots': {}}
{'archived_snapshots': {}, 'url': 'http://twitter.com/2ndHalfOnion\n'}
{'archived_snapshots': {}, 'url': 'http://twitter.com/2oliverbaker\n'}
{'archived_snapshots': {}, 'url': 'http://twitter.com/30ToMarsFandom\n'}
{'url': 'http://twitter.com/459Jisalge\n', 'archived_snapshots': {}}
{'archived_snapshots': {}, 'url': 'http://twitter.com/4claireevans\n'}
{'url': 'http://twitter.com/4ever1937\n', 'archived_snapshots': {'closest': {'status': '200', 'available': True, 'url': 'http://web.archive.org/web/20171003154

KeyboardInterrupt: ignored

# Scrape tweets from available caches

Once we've found the available archives we need to scrape the HTML of those pages for the tweet content. We'll use the `BeautifulSoup` library to do that.

Since the caches were taken at different times, the structure of the HTML may have changed. We'll need to write code that can handle parsing these different formats.

In [ ]:
import urllib
from bs4 import BeautifulSoup
import csv
import requests

In [ ]:
newTweetArr = []

## Testing scraping code

We've found two versions of the Twitter user pages in the caches. One from ~2015, and one used around ~2016-2017. We'll use one twitter page of each to test our parsing code, ensuring we get the data we want for each


In [ ]:
# testing for new version

url = "http://web.archive.org/web/20150603004258/https://twitter.com/AlwaysHungryBae"
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')

tweets = soup.find_all('li', attrs={'data-item-type': 'tweet'})

for t in tweets:
    tweet_obj = {}
    tweet_obj['tweet_id'] = t.get("data-item-id")
    tweet_container = t.find('div', attrs={'class': 'tweet'})
    tweet_obj['screen_name'] = tweet_container.get('data-screen-name')
    tweet_obj['permalink'] = tweet_container.get('data-permalink-path')
    tweet_content = tweet_container.find('p', attrs={'class': 'tweet-text'})
    tweet_obj['tweet_text'] = tweet_content.text
    tweet_obj['user_id'] = tweet_container.get('data-user-id')
    
    tweet_time = tweet_container.find('span', attrs={'class': '_timestamp'})
    tweet_obj['timestamp'] = tweet_time.get('data-time-ms')
    
    hashtags = tweet_container.find_all('a', attrs={'class': 'twitter-hashtag'})
    tweet_obj['hashtags'] = []
    tweet_obj['links'] = []
    
    for ht in hashtags:
        ht_obj = {}
        ht_obj['tag'] = ht.find('b').text
        ht_obj['archived_url'] = ht.get('href')
        tweet_obj['hashtags'].append(ht_obj)
    
    links = tweet_container.find_all('a', attrs={'class': 'twitter-timeline-link'})
    for li in links:
        li_obj = {}
        if li.get('data-expanded-url'):
            li_obj['url'] = li.get('data-expanded-url')
        elif li.get('data-resolved-url-large'):
            li_obj['url'] = li.get('data-resolved-url-large')
        else:
            li_obj['url'] = li.text
        li_obj['archived_url'] = li.get('href')
        tweet_obj['links'].append(li_obj)
    
    print(tweet_obj)

KeyboardInterrupt: ignored

In [ ]:

# testing for old version

url = "http://web.archive.org/web/20140615165707/https://twitter.com/NikaFast"
page = requests.get(url).text
soup = BeautifulSoup(page, 'html.parser')

#tweets = soup.find_all('p', attrs={'class': 'ProfileTweet-text'})
tweets = soup.find_all('div', attrs={'data-item-type': 'tweet'})


for t in tweets:
    #print(t.text)
    tweet_container = t.find('div')
    tweet_content = tweet_container.find('p', attrs={'class': 'ProfileTweet-text'}).text
    urls = []
    print("--------------------")
    for l in tweet_container.find_all('a', attrs={'class': 'twitter-timeline-link'}):
        print(l.get('href'))
        print(l.text)
    print(tweet_content)
    print(tweet_container.find('a', attrs={'class': 'js-permalink'}).get('href'))
    print(tweet_container.get('data-screen-name'))
    print(tweet_container.get('data-tweet-id'))
    print(tweet_container.get('data-user-id'))
    for h in tweet_container.find_all('a', attrs={'class': 'twitter-hashtag'}):
        print(h.get('href'))
        print(h.find('b').text)
    print("--------------------")

## Scraping Tweets

Now that our parsing code can handle either HTML format let's scrape all the tweets we can find from the available archives:

In [ ]:
tweet_arr = []
with open('/content/drive/MyDrive/Present work/2/avail_urls.txt') as f:
    #with open('tweets_2.csv', 'w') as tweet_file:
    #    writer  = csv.writer(tweet_file)
    for l in f:
        page = requests.get(l).text
        soup = BeautifulSoup(page, 'html.parser')
        
        tweets = soup.find_all('div', attrs={'data-item-type': 'tweet'})
        for t in tweets:
            tweet_container = t.find('div')
            try:
                tweet_content = tweet_container.find('p', attrs={'class': 'ProfileTweet-text'}).text
                tweet_obj = {}
                tweet_obj['tweet_text'] = tweet_content
                tweet_obj['permalink'] = tweet_container.find('a', attrs={'class': 'js-permalink'}).get('href')
                tweet_obj['screen_name'] = tweet_container.get('data-screen-name')
                tweet_obj['tweet_id'] = tweet_container.get('data-tweet-id')
                tweet_obj['user_id'] = tweet_container.get('data-user-id')
                tweet_obj['links'] = []
                tweet_obj['hashtags'] = []
                print("--------------------")
                for l in tweet_container.find_all('a', attrs={'class': 'twitter-timeline-link'}):
                    lo = {}
                    lo['archived_url'] = l.get('href')
                    lo['url'] = l.text
                    tweet_obj['links'].append(lo)
                for h in tweet_container.find_all('a', attrs={'class': 'twitter-hashtag'}):
                    ht = {}
                    ht['tag'] = h.find('b').text
                    ht['archived_url'] = h.get('href')
                    tweet_obj['hashtags'].append(ht)
                print("--------------------")
                tweet_arr.append(tweet_obj)
                print("processed a user")
            except:
                pass
        if not tweets:
            #newer html version
            tweets = soup.find_all('li', attrs={'data-item-type': 'tweet'})
            try:
                for t in tweets:
                    tweet_obj = {}
                    tweet_obj['tweet_id'] = t.get("data-item-id")
                    tweet_container = t.find('div', attrs={'class': 'tweet'})
                    tweet_obj['screen_name'] = tweet_container.get('data-screen-name')
                    tweet_obj['permalink'] = tweet_container.get('data-permalink-path')
                    tweet_content = tweet_container.find('p', attrs={'class': 'tweet-text'})
                    tweet_obj['tweet_text'] = tweet_content.text
                    tweet_obj['user_id'] = tweet_container.get('data-user-id')
                    
                    tweet_time = tweet_container.find('span', attrs={'class': '_timestamp'})
                    tweet_obj['timestamp'] = tweet_time.get('data-time-ms')

                    hashtags = tweet_container.find_all('a', attrs={'class': 'twitter-hashtag'})
                    tweet_obj['hashtags'] = []
                    tweet_obj['links'] = []

                    for ht in hashtags:
                        ht_obj = {}
                        ht_obj['tag'] = ht.find('b').text
                        ht_obj['archived_url'] = ht.get('href')
                        tweet_obj['hashtags'].append(ht_obj)

                    links = tweet_container.find_all('a', attrs={'class': 'twitter-timeline-link'})
                    for li in links:
                        li_obj = {}
                        if li.get('data-expanded-url'):
                            li_obj['url'] = li.get('data-expanded-url')
                        elif li.get('data-resolved-url-large'):
                            li_obj['url'] = li.get('data-resolved-url-large')
                        else:
                            li_obj['url'] = li.text
                        li_obj['archived_url'] = li.get('href')
                        tweet_obj['links'].append(li_obj)
                    tweet_arr.append(tweet_obj)
            except:
                pass
        if not tweets:
            print("NO TWEETS FOR " + l)

In [ ]:
# how many tweets did we find?
len(tweet_arr)

In [ ]:
# inspect a tweet
tweet_arr[-25]

In [ ]:
# write tweets to file
import json
with open('/content/drive/MyDrive/Present work/2/tweets_full.json', 'w') as f:
    json.dump(tweet_arr, f, ensure_ascii=False, sort_keys=True, indent=4)

# **Neo4j Import**

Now that we have our scraped tweet data we're ready to insert into Neo4j. We'll do this by loading the JSON data and passing it as a parameter to a Cypher query, using the Python driver for Neo4j.

In [ ]:
!pip install neo4j
from neo4j import GraphDatabase
import json

     |████████████████████████████████| 89 kB 1.1 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.0-py3-none-any.whl size=114860 sha256=c7922423b5c12c6178855466ee2fac80bfc71992d7ea0100aaaf068d5d7d655f
  Stored in directory: /root/.cache/pip/wheels/b8/cb/e4/d34e10b40d33911c281a6e9aa038d54ca7435310529b3f6042
Successfully built neo4j


In [ ]:
driver = GraphDatabase.driver("neo4j+s://1997272c.databases.neo4j.io:7687", auth=("neo4j", "DeTjqssc064er5wkA981vUaTyc09Jhz68m8WZ5WZq_k"))

In [ ]:
with open('/content/drive/MyDrive/Sem 7/Mini project/Code/Data/tweets_3(2)-3.json') as json_data:
    tweetArr = json.load(json_data)

In [ ]:
len(tweetArr)

235

# Original query:
```
import_query = '''
WITH $tweetArr AS tweets
UNWIND tweets AS tweet
MERGE (u:User {user_id: tweet.user_id})
ON CREATE SET u.screen_name = tweet.screen_name
MERGE (t:Tweet {tweet_id: tweet.tweet_id})
ON CREATE SET t.text = tweet.tweet_text,
              t.permalink = tweet.permalink
MERGE (u)-[:POSTED]->(t)

FOREACH (ht IN tweet.hashtags |
  MERGE (h:Hashtag {tag: ht.tag })
  ON CREATE SET h.archived_url = ht.archived_url
  MERGE (t)-[:HAS_TAG]->(h)
)

FOREACH (link IN tweet.links |
  MERGE (l:Link {url: link.url})
  ON CREATE SET l.archived_url = link.archived_url
  MERGE (t)-[:HAS_LINK]->(l)
)

'''

```




In [ ]:
import_query = '''
WITH $tweetArr AS tweets
UNWIND tweets AS tweet
MERGE (u:User {user_id: tweet.user_id})
ON CREATE SET u.screen_name = tweet.screen_name
MERGE (t:Tweet {tweet_id: tweet.tweet_id})
ON CREATE SET t.text = tweet.tweet_text,
              t.permalink = tweet.permalink
MERGE (u)-[:POSTED]->(t)

FOREACH (ht IN tweet.hashtags |
  MERGE (h:Hashtag {tag: ht.tag })
  ON CREATE SET h.archived_url = ht.archived_url
  MERGE (t)-[:HAS_TAG]->(h)
)

FOREACH (link IN tweet.links |
  MERGE (l:Link {url: link.url})
  ON CREATE SET l.archived_url = link.archived_url
  MERGE (t)-[:HAS_LINK]->(l)
)

'''

def add_tweets(tx, tweetArr):
    tx.run(import_query, tweetArr=tweetArr)

print (driver)

In [ ]:
with driver.session() as session:
    session.write_transaction(add_tweets, tweetArr)

Transaction failed and will be retried in 1.0710115601539212s (The allocation of an extra 125.7 MiB would use more than the limit 250.0 MiB. Currently using 221.2 MiB. dbms.memory.transaction.global_max_size threshold reached)
Transaction failed and will be retried in 2.101121429884486s (The allocation of an extra 125.7 MiB would use more than the limit 250.0 MiB. Currently using 221.2 MiB. dbms.memory.transaction.global_max_size threshold reached)


KeyboardInterrupt: ignored

In [ ]:
driver.close()